In [1]:
import os
import pandas as pd
import json
from IPython.display import display, Markdown, Latex
from tqdm.auto import tqdm

from common.consts import RESULTS_DIR, EVAL_SIZE
from common.utils import filename_to_obj, remove_index

/home/tdolega/miniconda/envs/p311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def results_as_pandas(filename):
    path = os.path.join(RESULTS_DIR, filename)
    with open(path, "r") as f:
        data = f.readlines()
    data = [json.loads(d) for d in data]
    data = pd.DataFrame(data)

    if len(data) == 0:
        print(f"empty file: {filename}")
        return data

    params = filename_to_obj(filename)
    for k, v in params.items():
        data[k] = v

    data = data.explode("evaluations")
    data = data.rename_axis("question_idx").reset_index()

    data = pd.concat([data, data["evaluations"].apply(pd.Series)], axis=1)
    evaluation_keys = data["evaluations"].apply(pd.Series).columns
    for col in evaluation_keys:
        data = pd.concat([data, data[col].apply(pd.Series).add_prefix(f"{col}/")], axis=1)
        data = data.drop(columns=col)
    data = data.drop(columns=["evaluations"])

    return data


files = os.listdir(RESULTS_DIR)
params_names = list(filename_to_obj(files[0]).keys())
all_results = pd.concat([results_as_pandas(f) for f in tqdm(files)])
all_results.head()

100%|██████████| 27/27 [00:14<00:00,  1.90it/s]


,question_idx,question_id,llm,prompt_id,temperature,nli,ellm,sim,citations/ais_recall,citations/ais_precision,...,citations/supported,citations/citations,citations/correct_citations,citations/out_of_range,correctness/answer_overlap,correctness/answer_entail,correctness/citations_recall,correctness/citations_precision,quality/answer_relevance,quality/new_question
0,0,5abab42e55429955dce3eed2,gpt-3.5-turbo-0125,1,0.1,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,0.0,1.0,...,"[0, 0]","[[], [3]]","[[], [True]]","[0, 0]",0.5,0.0,0.5,1.0,0.730736,What is the relationship between Jordan Subban...
1,0,5abab42e55429955dce3eed2,gpt-3.5-turbo-0125,1,0.1,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,0.0,1.0,...,"[0, 0]","[[], [3]]","[[], [True]]","[0, 0]",0.5,0.0,0.5,1.0,0.822013,What professional hockey player was drafted fi...
2,0,5abab42e55429955dce3eed2,gpt-3.5-turbo-0125,1,0.1,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,0.0,1.0,...,"[0, 0]","[[], [3]]","[[], [True]]","[0, 0]",0.5,0.0,0.5,1.0,0.730736,What is the relationship between Jordan Subban...
3,1,5a761900554299109176e648,gpt-3.5-turbo-0125,1,0.1,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,1.0,1.0,...,[1],[[1]],[[True]],[0],1.0,1.0,0.5,1.0,0.965509,What is the name of the lobbying group that wa...
4,1,5a761900554299109176e648,gpt-3.5-turbo-0125,1,0.1,t5_xxl_true_nli_mixture,Mistral-7B-Instruct-v0.2,all-MiniLM-L6-v2,1.0,1.0,...,[1],[[1]],[[True]],[0],1.0,1.0,0.5,1.0,0.974511,What is the name of the lobbying group that wa...


In [3]:
all_obj_cols = all_results.select_dtypes(include=["object"]).columns
drop_obj_cols = list(set(all_obj_cols) - set(params_names))
drop_obj_cols.remove("question_id")
print(f"Dropping columns: {drop_obj_cols}")
all_num_results = all_results.drop(columns=drop_obj_cols)

Dropping columns: ['citations/correct_citations', 'citations/sentences', 'quality/new_question', 'citations/citations', 'citations/supported', 'citations/out_of_range']


In [4]:
eval_split = all_num_results[all_num_results["question_idx"] < EVAL_SIZE]
train_split = all_num_results[all_num_results["question_idx"] >= EVAL_SIZE]

In [5]:
def aggregate(split):
    split = split.drop(columns=["question_idx"])
    results_with_std_for_each_question = split.groupby([*params_names, "question_id"]).agg(["mean", "std"])
    results_for_each_model = results_with_std_for_each_question.groupby(params_names)
    results = results_for_each_model.mean()
    results["n_questions"] = results_for_each_model.size()
    return results


eval_results = aggregate(eval_split)
train_results = aggregate(train_split)

if eval_results["n_questions"].nunique() != 1:
    print("Warning: not all rows in evaluation have the same number of examples")

In [6]:
display(Markdown("### Prompts comparison"))
parameter_results = eval_results[eval_results.index.get_level_values("llm") == "Mistral-7B-Instruct-v0.2"]
parameter_results[parameter_results.index.get_level_values("temperature") == "0.1"]

### Prompts comparison

citations/ais_recall  \
                                                                                                                                 mean   
llm                      prompt_id temperature nli                     ellm                     sim                                     
Mistral-7B-Instruct-v0.2 1         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.575644   
                         2         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.501540   
                         3         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.580222   

                                                                                                                            \
                                                                                                                       std   
llm                      prompt_id temperature nli                     ellm                     sim                          
Mistral-7B-Instruct-v0.2 1         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.036560   
                         2         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.066146   
                         3         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.045033   

                                                                                                                 citations/ais_precision  \
                                                                                                                                    mean   
llm                      prompt_id temperature nli                     ellm                     sim                                        
Mistral-7B-Instruct-v0.2 1         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2                0.871517   
                         2         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2                0.862167   
                         3         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2                0.890076   

                                                                                                                            \
                                                                                                                       std   
llm                      prompt_id temperature nli                     ellm                     sim                          
Mistral-7B-Instruct-v0.2 1         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.038350   
                         2         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.034978   
                         3         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.024787   

                                                                                                                 citations/n_sentences  \
                                                                                                                                  mean   
llm                      prompt_id temperature nli                     ellm                     sim                                      
Mistral-7B-Instruct-v0.2 1         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2              2.573333   
                         2         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2              2.473333   
                         3         0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2              1.980000   

                                                                                                          

In [7]:
display(Markdown("### Temperature comparison"))
parameter_results[parameter_results.index.get_level_values("prompt_id") == "1"]

### Temperature comparison

citations/ais_recall  \
                                                                                                                                 mean   
llm                      prompt_id temperature nli                     ellm                     sim                                     
Mistral-7B-Instruct-v0.2 1         0.01        t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.574825   
                                   0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.575644   
                                   0.2         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.544493   
                                   0.3         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.566712   
                                   0.4         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.580572   
                                   0.5         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.569651   
                                   0.6         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.519373   
                                   0.7         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.555853   
                                   0.8         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.505895   
                                   0.9         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.572745   
                                   1.0         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.550218   
                                   1.5         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.524532   
                                   2.0         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.463269   

                                                                                                                            \
                                                                                                                       std   
llm                      prompt_id temperature nli                     ellm                     sim                          
Mistral-7B-Instruct-v0.2 1         0.01        t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.027143   
                                   0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.036560   
                                   0.2         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.081991   
                                   0.3         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.084729   
                                   0.4         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.150051   
                                   0.5         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.144597   
                                   0.6         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.168079   
                                   0.7         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.182757   
                                   0.8         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.195393   
                                   0.9         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.204617   
                                   1.0         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.202885   
                                   1.5         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.221513   
                              

In [8]:
display(Markdown("### Evaluation results"))
eval_display = eval_results[eval_results.index.get_level_values("prompt_id") == "1"]
eval_display = eval_display[eval_display.index.get_level_values("temperature") == "0.1"]
eval_display = remove_index(eval_display, "prompt_id")
eval_display = eval_display.sort_values(by=("correctness/citations_recall", "mean"), ascending=False)
eval_display

### Evaluation results

prompt_id  \
                                                                                                                          
llm                             temperature nli                     ellm                     sim                          
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gpt-3.5-turbo-0125              0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mixtral-8x7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gpt-4-turbo                     0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Meta-Llama-3-70B-Instruct       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Meta-Llama-3-8B-Instruct        0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mistral-7B-Instruct-v0.2        0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
zephyr-7b-beta                  0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Phi-3-mini-4k-instruct          0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
tinyllama-1.1b-chat-v1.0.Q8_0   0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
gemma-1.1-2b-it                 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
Mistral-7B-Instruct-v0.1        0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   

                                                                                                              citations/ais_recall  \
                                                                                                                              mean   
llm                             temperature nli                     ellm                     sim                                     
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.604452   
gpt-3.5-turbo-0125              0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.737556   
Mixtral-8x7B-Instruct-v0.1      0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.653500   
gpt-4-turbo                     0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.733013   
Meta-Llama-3-70B-Instruct       0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.691944   
Meta-Llama-3-8B-Instruct        0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.492702   
Mistral-7B-Instruct-v0.2        0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.575644   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.528167   
zephyr-7b-beta                  0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.406690   
Phi-3-mini-4k-instruct          0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.252796   
tinyllama-1.1b-chat-v1.0.Q8_0   0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.041361   
gemma-1.1-2b-it                 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.006667   
Mistral-7B-Instruct-v0.1        0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2             0.005000 

In [9]:
def show_cleaned_results(short_eval_display):
    short_eval_display = remove_index(short_eval_display, "temperature")
    short_eval_display = remove_index(short_eval_display, "nli")
    short_eval_display = remove_index(short_eval_display, "ellm")
    short_eval_display = remove_index(short_eval_display, "sim")
    important_columns = ["citations/ais_recall", "citations/ais_precision", "correctness/answer_overlap", "correctness/answer_entail", "correctness/citations_recall", "correctness/citations_precision", "quality/answer_relevance"]
    short_eval_display = short_eval_display[important_columns]
    return short_eval_display


show_cleaned_results(eval_display)

citations/ais_recall            \
                                                mean       std   
llm                                                              
mixtral-8x7b-instruct-v0.1.Q8_0             0.604452  0.084171   
gpt-3.5-turbo-0125                          0.737556  0.072604   
Mixtral-8x7B-Instruct-v0.1                  0.653500  0.061199   
gpt-4-turbo                                 0.733013  0.045401   
Meta-Llama-3-70B-Instruct                   0.691944  0.049556   
Meta-Llama-3-8B-Instruct                    0.492702  0.060547   
Mistral-7B-Instruct-v0.2                    0.575644  0.036560   
qwen1_5-110b-chat                           0.528167  0.095486   
zephyr-7b-beta                              0.406690  0.090558   
Phi-3-mini-4k-instruct                      0.252796  0.030503   
tinyllama-1.1b-chat-v1.0.Q8_0               0.041361  0.010585   
gemma-1.1-2b-it                             0.006667  0.005774   
Mistral-7B-Instruct-v0.1                    0.005000  0.000000   

                                citations/ais_precision            \
                                                   mean       std   
llm                                                                 
mixtral-8x7b-instruct-v0.1.Q8_0                0.921198  0.053556   
gpt-3.5-turbo-0125                             0.995833  0.002887   
Mixtral-8x7B-Instruct-v0.1                     0.928603  0.029692   
gpt-4-turbo                                    0.985000  0.005774   
Meta-Llama-3-70B-Instruct                      0.955556  0.017802   
Meta-Llama-3-8B-Instruct                       0.951210  0.022033   
Mistral-7B-Instruct-v0.2                       0.871517  0.038350   
qwen1_5-110b-chat                              0.976111  0.026943   
zephyr-7b-beta                                 0.910524  0.068665   
Phi-3-mini-4k-instruct                         0.578455  0.069580   
tinyllama-1.1b-chat-v1.0.Q8_0                  0.125000  0.031754   
gemma-1.1-2b-it                                0.046111  0.009623   
Mistral-7B-Instruct-v0.1                       0.010000  0.000000   

                                correctness/answer_overlap            \
                                                      mean       std   
llm                                                                    
mixtral-8x7b-instruct-v0.1.Q8_0                   0.882598  0.007698   
gpt-3.5-turbo-0125                                0.884244  0.012990   
Mixtral-8x7B-Instruct-v0.1                        0.909923  0.001443   
gpt-4-turbo                                       0.877302  0.007661   
Meta-Llama-3-70B-Instruct                         0.865923  0.020651   
Meta-Llama-3-8B-Instruct                          0.917846  0.000000   
Mistral-7B-Instruct-v0.2                          0.872812  0.014915   
qwen1_5-110b-chat                                 0.867315  0.020207   
zephyr-7b-beta                                    0.853718  0.028674   
Phi-3-mini-4k-instruct                            0.711068  0.012317   
tinyllama-1.1b-chat-v1.0.Q8_0                     0.566297  0.028318   
gemma-1.1-2b-it                                   0.630654  0.038105   
Mistral-7B-Instruct-v0.1                          0.790410  0.014434   

                                correctness/answer_entail            \
                                                     mean       std   
llm                                                                   
mixtral-8x7b-instruct-v0.1.Q8_0                  0.856667  0.023094   
gpt-3.5-turbo-0125                               0.910000  0.011547   
Mixtral-8x7B-Instruct-v0.1                       0.896667  0.011547   
gpt-4-turbo                                      0.900000  0.011547   
Meta-Llama-3-70B-Instruct                        0.873333  0.017321   
Meta-Llama-3-8B-Instruct                         0.903333  0.005774   
Mistral-7B-Instruct-v0.2                         0.886667  0.017321   
qwen1_5

In [10]:
display(Markdown("### Training results"))
train_display = remove_index(train_results, "prompt_id")
train_display

### Training results

prompt_id  \
                                                                                                                          
llm                             temperature nli                     ellm                     sim                          
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2         1   

                                                                                                              citations/ais_recall  \
                                                                                                                              mean   
llm                             temperature nli                     ellm                     sim                                     
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2              0.62231   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2              0.56492   

                                                                                                                         \
                                                                                                                    std   
llm                             temperature nli                     ellm                     sim                          
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.075736   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.117130   

                                                                                                              citations/ais_precision  \
                                                                                                                                 mean   
llm                             temperature nli                     ellm                     sim                                        
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2                0.901109   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2                0.964120   

                                                                                                                         \
                                                                                                                    std   
llm                             temperature nli                     ellm                     sim                          
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.037999   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2  0.030177   

                                                                                                              citations/n_sentences  \
                                                                                                                               mean   
llm                             temperature nli                     ellm                     sim                                      
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2              2.654258   
qwen1_5-110b-chat               0.1         t5_xxl_true_nli_mixture Mistral-7B-Instruct-v0.2 all-MiniLM-L6-v2              1.936333   

                                                                                                                         \
                                                                                                          